# `nanoGPT`

## Install / Setup

### First Time Running

We need to install `ngpt` and setup the Shakespeare dataset

This will need to be ran the first time you are running this notebook.

Following the

```python
!python3 -m pip install nanoGPT
```

you will need to restart your runtime (Runtime -> Restart runtime)

After this, you should be able to

```python
>>> import ngpt
>>> ngpt.__file__
'/content/nanoGPT/src/ngpt/__init__.py'
```

In [1]:
%%bash

[ $(python3 -c 'import ngpt; print(ngpt.__file__)') ] && export HAS_NGPT=1 || export HAS_NGPT=0
[ $HAS_NGPT ] || git clone https://github.com/saforem2/nanoGPT;
[ $HAS_NGPT ] || python3 nanoGPT/data/shakespeare_char/prepare.py
[ $HAS_NGPT ] || python3 -m pip install -e nanoGPT -vvv

## Post Install

If installed correctly, you should be able to:

```python
>>> import ngpt
>>> ngpt.__file__
'/path/to/nanoGPT/src/ngpt/__init__.py'
```

In [2]:
%load_ext autoreload
%autoreload 2

import ngpt
from rich import print
print(ngpt.__file__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py

## Build Trainer

Explicitly, we:

1. `setup_torch(...)`
2. Build `cfg: DictConfig = get_config(...)`
3. Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4. Build `trainer = Trainer(config)`

In [3]:
import os
from ezpz import setup_torch
from hydra.utils import instantiate
from ngpt.configs import get_config
from ngpt.trainer import Trainer

os.environ['MASTER_PORT'] = '4235'
rank = setup_torch('DDP', seed=1234)
cfg = get_config(
    [
        'data=owt',              # open web text
        'model=gpt2',            # gpt2 arch.
        'optimizer=gpt2',
        'train=gpt2',
        'train.init_from=gpt2',  # init from GPT2
        'train.max_iters=1000',
        'train.dtype=bfloat16',
    ]
)
config = instantiate(cfg)
trainer = Trainer(config)

--------------------------------------------------------------------------

  Local host:   thetagpu24
  Local device: mlx5_0
--------------------------------------------------------------------------
RANK: 0 / 0


[2023-11-10 15:26:45][WARNING][configs.py:298] - No meta.pkl found, assuming GPT-2 encodings...
[2023-11-10 15:26:45][INFO][configs.py:264] - Rescaling GAS -> GAS // WORLD_SIZE = 1 // 1
[2023-11-10 15:26:45][INFO][configs.py:399] - Tokens per iteration: 12,288
[2023-11-10 15:26:45][INFO][configs.py:431] - Using <torch.amp.autocast_mode.autocast object at 0x7fd7f13aee00>
[2023-11-10 15:26:45][INFO][trainer.py:184] - Initializing from OpenAI GPT-2 Weights: gpt2


2023-11-10 15:26:45.538583: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[2023-11-10 15:26:49,447] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-11-10 15:26:50][INFO][model.py:225] - loading weights from pretrained gpt: gpt2
[2023-11-10 15:26:50][INFO][model.py:234] - forcing vocab_size=50257, block_size=1024, bias=True
[2023-11-10 15:26:50][INFO][model.py:240] - overriding dropout rate to 0.0
[2023-11-10 15:26:52][INFO][model.py:160] - number of parameters: 123.65M
[2023-11-10 15:26:56][INFO][model.py:290] - num decayed parameter tensors: 50, with 124,318,464 parameters
[2023-11-10 15:26:56][INFO][model.py:291] - num non-decayed parameter tensors: 98, with 121,344 parameters
[2023-11-10 15:26:56][INFO][model.py:297] - using fused AdamW: True


Process ForkProcess-11:
Process ForkProcess-15:
Process ForkProcess-12:
Process ForkProcess-8:
Process ForkProcess-22:
Process ForkProcess-29:
Process ForkProcess-3:
Process ForkProcess-28:
Process ForkProcess-7:
Process ForkProcess-13:
Process ForkProcess-17:
Process ForkProcess-2:
Process ForkProcess-10:
Process ForkProcess-24:
Process ForkProcess-30:
Process ForkProcess-25:
Process ForkProcess-6:
Process ForkProcess-1:
Process ForkProcess-16:
Process ForkProcess-20:
Process ForkProcess-9:
Process ForkProcess-26:
Process ForkProcess-27:
Process ForkProcess-32:
Process ForkProcess-4:
Process ForkProcess-23:
Process ForkProcess-18:
Process ForkProcess-14:
Process ForkProcess-19:
Process ForkProcess-21:
Process ForkProcess-5:
Process ForkProcess-31:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/lus/theta-fs0/software/thetagpu/conda/2023-01-11/mconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
  

## Train Model

Legend:

<div style="text-align:left;">

|  **NAME**  |     **DESCRIPTION**          |
|:----------:|:----------------------------:|
|   `step`   | Current training step        |
|   `loss`   | Loss value                   |
|   `dt`     | Time per step (in **ms**)    |
|   `sps`    | Samples per second           |
|   `mtps`   | (million) Tokens per sec     |
|   `mfu`    | Model Flops Utilization*     |

*in units of A100 `bfloat16` peak FLOPS

</div>

In [4]:
trainer.train()

  0%|          | 0/1000 [00:00<?, ?it/s]

[2023-11-10 15:29:47][INFO][trainer.py:540] - step=100 loss=3.218 dt=123.690 sps=8.085 mtps=0.099 mfu=27.230 train_loss=3.094 val_loss=3.108
[2023-11-10 15:29:59][INFO][trainer.py:540] - step=200 loss=2.946 dt=123.632 sps=8.089 mtps=0.099 mfu=27.231 train_loss=3.094 val_loss=3.108
[2023-11-10 15:30:12][INFO][trainer.py:540] - step=300 loss=3.171 dt=123.695 sps=8.084 mtps=0.099 mfu=27.231 train_loss=3.094 val_loss=3.108
[2023-11-10 15:30:24][INFO][trainer.py:540] - step=400 loss=2.858 dt=123.382 sps=8.105 mtps=0.100 mfu=27.237 train_loss=3.094 val_loss=3.108
[2023-11-10 15:30:37][INFO][trainer.py:540] - step=500 loss=2.919 dt=123.775 sps=8.079 mtps=0.099 mfu=27.235 train_loss=3.094 val_loss=3.108
[2023-11-10 15:30:49][INFO][trainer.py:540] - step=600 loss=3.097 dt=123.666 sps=8.086 mtps=0.099 mfu=27.235 train_loss=3.094 val_loss=3.108
[2023-11-10 15:31:02][INFO][trainer.py:540] - step=700 loss=3.034 dt=123.714 sps=8.083 mtps=0.099 mfu=27.234 train_loss=3.094 val_loss=3.108
[2023-11-10 1

## Evaluate Model

In [13]:
query = "What is a supercomputer?"
outputs = trainer.evaluate(query, num_samples=1, display=False)

In [18]:
from rich.text import Text
from enrich.console import get_console
console = get_console()

console.print(fr'\[prompt]: "{query}"')
console.print("\[response]:\n\n" + fr"{outputs['0']['raw']}")

[prompt]: "What is a supercomputer?"

[response]:

What is a supercomputer? A supercomputer is a machine that is capable of using any available computing talent while being clever at its job.

What is an artificial intelligence? A computer program that's built using data from an existing machine.

What are the advantages of artificial intelligence? A computer can perform some scientific work while not necessarily being intelligent.

What are the advantages of artificial intelligence? A computer can perform some scientific work while not necessarily being intelligent.

What are the advantages of artificial intelligence? A computer can perform some scientific work while not necessarily being intelligent.

What is the cost of artificial intelligence? It costs enormous sums of money to make a supercomputer. Every additional $10,000 to $20,000 can be invested into make the machine smarter. A supercomputer can be used to do that work while not being intelligent.

What are supercomputer strengths? A computer is capable of supercomputers.

To learn more about artificial intelligence and how Google can use it, click here to visit our blog.

Image credit: Lyle McKee, The Times<|endoftext|>Image copyright EPA Image caption The UK is planning to double the size of stores for the first time in four years

The UK is planning to double the size of stores in the UK by 2020, according to government plans.

The announcement of a four-year plan to double the size of stores in the UK was announced ahead of the Home Office's launch on Wednesday.

The plans will take effect on 1 December.

The first 100 stores will be open by 2020, with a further 300 stores by 2020.

The plans, announced by the Prime Minister David Cameron at the start of the 2017/18 Budget, will be announced later this week.

They include:

A 20% reduction in the size of the Sainsbury's supermarket chain immediately following a first year of surplus levels

A 100% reduction in the size of the supermarket chain and the Sainsbury's in the UK.

The policy also sets out "more concrete and immediate action" aimed at reducing the size of the UK's supermarkets, the Prime Minister announced on Wednesday.

The size of the UK's supermarkets has been measured by the size of the UK's total retail market on average.

Image copyright Getty Images

The British government released the plans, which were part of the Conservative manifesto, in a separate draft in April, allowing the US

Exception ignored in atexit callback: <bound method InteractiveShell.atexit_operations of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7fd7eeacd510>>

Traceback (most recent call last):

File "/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/venvs/thetaGPU/2023-01-11/lib/python3.10/site-packages/rich/jupyter.py", line 91, in display

...

File "/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/venvs/thetaGPU/2023-01-11/lib/python3.10/site-packages/rich/console.py", line 860, in __enter__

self._enter_buffer()

File "/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/venvs/thetaGPU/2023-01-11/lib/python3.10/site-packages/rich/console.py", line 818, in _enter_buffer

self._buffer_index += 1

RecursionError: maximum recursion depth exceeded